In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install() # expect a kernel restart

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:25
🔁 Restarting kernel...


In [ ]:
%cd .
%cd /content

/content
/content


In [ ]:
! git clone https://github.com/HenryPengZou/ImplicitAVE.git

Cloning into 'ImplicitAVE'...
remote: Enumerating objects: 2089, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 2089 (delta 91), reused 60 (delta 45), pack-reused 1962 (from 1)
Receiving objects: 100% (2089/2089), 79.94 MiB | 42.92 MiB/s, done.
Resolving deltas: 100% (201/201), done.


In [ ]:
%cd ImplicitAVE

/content/ImplicitAVE


In [ ]:
!conda create -n Qwen python=3.9 -y

Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/Qwen

  added / updated specs:
    - python=3.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bzip2-1.0.8                |       h4bc722e_7         247 KB  conda-forge
    ca-certificates-2024.8.30  |       hbcca054_0         155 KB  conda-forge
    ld_impl_linux-64-2.43      |       h712a8e2_2         654 KB  conda-forge
    libgcc-14.2.0              |       h77fa898_1         829 KB  conda-forge
    libgcc-ng-14.2.0           |       h69a702a_1          53 KB  conda-forge
    libgomp-14.2.0             |       h77fa898_1         450 KB  conda-forge
    lib

In [ ]:
!source activate Qwen

In [ ]:
!conda install pytorch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 pytorch-cuda=11.8 -c pytorch -c nvidia


Channels:
 - pytorch
 - nvidia
 - conda-forge
Platform: linux-64
Solving environment: | / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pytorch-cuda=11.8
    - pytorch==2.2.2
    - torchaudio==2.2.2
    - torchvision==0.17.2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |       2_kmp_llvm           6 KB  conda-forge
    blas-2.116                 |              mkl          13 KB  conda-forge
    blas-devel-3.9.0           |   16_linux64_mkl          12 KB  conda-forge
    certifi-2024.8.30          |     pyhd8ed1ab_0         160 KB  conda-forge
    cuda-cudart-11.8.89        |                0         197 

In [ ]:
%cd code/Qwen_VL
!pip install -r requirements.txt


In [ ]:
# Set the number of samples to be used for evaluation;
# n_evaluation_samples = 3 # 3 samples are used for evaluation
n_evaluation_samples = -1 # all samples are used for evaluation

In [ ]:
# Move to directory with data
%cd ../../data/evaluation_data

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
from sklearn.metrics import f1_score, accuracy_score
import torch
import numpy as np
import random
import gc
import os
import json
import pickle as pkl
import pandas as pd

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL", trust_remote_code=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL", device_map="cuda", trust_remote_code=True).eval()

In [ ]:
def preprocess(df, name_to_value):
  id_to_prompt = {}
  id_to_label = {}

  df['prompt'] = df.apply(lambda row: f"What is the {row['attribute_names']} of this product: {row['texts']}? Answer with the option from the given choices directly: {name_to_value[row['attribute_names']]}.\nAnswer:", axis=1)


  for row in df.itertuples():
    id_to_prompt[row.id] = row.prompt
    id_to_label[row.id] = row.attribute_values

  return id_to_prompt, id_to_label

In [ ]:
def extract_answer(text):
    # Look for the pattern " Answer:" and extract the text immediately following it until the next quotation mark
    start_keyword = "Answer:"
    end_char = '"'

    # Find the starting position of the answer
    start_index = text.find(start_keyword)
    if start_index == -1:
        # If " Answer:" is not found, return an empty string or an error message
        return "Answer not found"

    # Adjust start_index to get the actual start of the answer text
    start_index += len(start_keyword) + 1

    # Find the end of the answer based on the next quotation mark
    end_index = text.find(end_char, start_index)
    if end_index == -1:
        # If the closing quotation mark is not found, return the substring from the start_index to the end of the string
        return text[start_index:]

    # Extract and return the answer
    return text[start_index:end_index]

In [ ]:
def inference(model, tokenizer, id_to_prompts, id_to_labels, img_dir):
  id_to_pred = {}
  preds = []
  labels = []

  for id in id_to_prompts:
    labels.append(id_to_labels[id])
    text = id_to_prompts[id]
    img_name = f'{id}.jpg'
    img_path = os.path.join(img_dir, img_name)

    if not os.path.exists(img_path):
      continue

    query = tokenizer.from_list_format([
    {'image': img_path},
    {'text': text},
      ])
    inputs = tokenizer(query, return_tensors='pt')
    inputs = inputs.to(model.device)
    pred = model.generate(**inputs)
    out = tokenizer.decode(pred.cpu()[0], skip_special_tokens=False)

    output = extract_answer(out)[:-13]


    preds.append(output)
    id_to_pred[id] = output

    torch.cuda.empty_cache()
    gc.collect()

  return id_to_pred, preds, labels

In [ ]:
def calculate_metrics(predictions, targets):
  binary_predictions = [1 if target.lower() in prediction.lower() else 0 for target, prediction in zip(targets, predictions)]
  binary_targets = [1] * len(targets)

  accuracy = sum(binary_predictions)
  micro_f1 = f1_score(binary_targets, binary_predictions, average='micro')
  return accuracy, micro_f1

In [ ]:
file_names = [
    'Clothing_annotated_final.tsv',
    'Jewlery_and_General_Apparel_annotated_final.tsv',
    'Footwear_annotated_final.tsv',
    'Food_annotated_final.tsv',
    'Home_annotated_final.tsv'
]

In [ ]:
img_dir = "images"
for dataset in file_names:
  if "Food" in dataset:
    with open("options_Food.json", "r") as f:
      name_to_val = json.load(f)

  elif "Home" in dataset:
    with open("options_Home.json", "r") as f:
      name_to_val = json.load(f)

  else:
    with open("options_Clothing_Shoes_and_Jewelry.json", "r") as f:
      name_to_val = json.load(f)

  data = pd.read_csv(f'texts/{dataset}', sep='\t')[:n_evaluation_samples]

  id_to_prompts, id_to_labels = preprocess(data, name_to_val)
  id_to_pred, preds, labels = inference(model, tokenizer, id_to_prompts, id_to_labels, img_dir)
  both_acc, both_micro_f1 = calculate_metrics(preds, labels)

  import os

  # Ensure the directory exists
  os.makedirs('results', exist_ok=True)

  # Save predictions
  with open(f'results/Qwen_Vl_7B_preds_both_{dataset[:-4]}.pkl', 'wb') as f:
    pkl.dump(id_to_pred, f)

  print(f'For {dataset[:-4]} multi modalities micro_f1 was {both_micro_f1}')
  print('Prediction Examples: ', preds[:3])
  print('Label Examples: ', labels[:3])

  torch.cuda.empty_cache()
  gc.collect()

NameError: name 'file_names' is not defined

Prompt Testing Code

In [ ]:
def preprocess(df, name_to_value, custom_prompt):
    id_to_prompt = {}
    id_to_label = {}

    def apply_custom_prompt(row):
        return custom_prompt.format(
            attribute_names=row['attribute_names'],
            category=row['category'],
            texts=row['texts'],
            options=name_to_value[row['attribute_names']]
        )

    df['prompt'] = df.apply(apply_custom_prompt, axis=1)

    for row in df.itertuples():
        id_to_prompt[row.id] = row.prompt
        id_to_label[row.id] = row.attribute_values

    return id_to_prompt, id_to_label

In [ ]:
custom_prompts = [
    "Question: What is {attribute_names} of this product?\nContext: [Category] {category} {texts}.\nYou must only answer the question with exactly one of the following options {options}.\nAnswer:",
    "What is {attribute_names} of this product?[Category] {category} {texts}.Answer with the option from the given choices directly: {options}.\nAnswer:",
    "[Category] {category} {texts}. What is {attribute_names} of this product? Answer with the option from the given choices directly: {options}.\nAnswer:",
    "[Category] {category} {texts}. What is {attribute_names} of this product based on the given information and the given image? Answer with the option from the given choices directly: {options}.\nAnswer:",
    "[Category] {category} {texts}. Which one of {options} is the {attribute_names} of this product? Answer with the option from the given choices directly.\nAnswer:",
    "{texts}. What is the {attribute_names} of this product? Answer with the option from the given choices directly: {options}.\nAnswer:",
    "{texts}. Based on the description and the image, what is the {attribute_names} of this product? Answer with the option from the given choices directly: {options}.\nAnswer:",
    "What is the {attribute_names} of this product: {texts}? Answer with the option from the given choices directly: {options}.\nAnswer:"
]

In [ ]:
img_dir = "images"
dataset = "texts/Clothing_annotated_final.tsv"
with open("options_Clothing_Shoes_and_Jewelry.json", "r") as f:
    name_to_val = json.load(f)

for i, prompt in enumerate(custom_prompts):
  data = pd.read_csv(f'{dataset}', sep='\t')[:n_evaluation_samples]

  id_to_prompts, id_to_labels = preprocess(data, name_to_val, prompt)
  id_to_pred, preds, labels = inference(model, tokenizer, id_to_prompts, id_to_labels, img_dir)
  both_acc, both_micro_f1 = calculate_metrics(preds, labels)


  print(f'For prompt {i} the micro_f1 was {both_micro_f1}')

  torch.cuda.empty_cache()
  gc.collect()